In [91]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from functools import partial

In [92]:
df = pd.read_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/train.csv')
corr = pd.read_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/train_updates_20220929.csv')
corr_ids = list(corr['seq_id'])
df = df[~df['seq_id'].isin(corr_ids)]

In [93]:
print(df['tm'].max())

130.0


In [94]:
wild_type = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'
print(df.head())
letters = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

   seq_id                                   protein_sequence   pH  \
0       0  AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0   
1       1  AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0   
2       2  AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0   
3       3  AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0   
4       4  AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...  7.0   

                         data_source    tm  
0  doi.org/10.1038/s41592-020-0801-4  75.7  
1  doi.org/10.1038/s41592-020-0801-4  50.5  
2  doi.org/10.1038/s41592-020-0801-4  40.5  
3  doi.org/10.1038/s41592-020-0801-4  47.2  
4  doi.org/10.1038/s41592-020-0801-4  49.5  


In [95]:
def similar(s, c, w, b, side=None): 
    if side == 'start':
        return SequenceMatcher(None, s[c][:b], w).ratio()
    elif side == 'end':
        return SequenceMatcher(None, s[c][-b:], w).ratio()
    elif side == None:
        return SequenceMatcher(None, s[c][:], w).ratio()
    else:
        raise ValueError('side must be `start`, `end`, or None')
        
def match_columns(wild, train):
    bounds = [1, 3, 5, 15, 30]
    for bound in bounds:
        start_wild = wild[:bound]
        end_wild = wild[-bound:]
        train[f'first{bound}'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound, side='start'), axis=1)
        train[f'last{bound}'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound, side='end'), axis=1)
    train['sim_ratio'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound), axis=1)
    return train

def tenth_strings(s, c, num, tenth):
    return s[c][num*s[tenth] : (num+1)*s[tenth]]

def tenths_letters(train, letters):
    col = train['protein_sequence']
    length = len(col)
    tenths = [int((length/25)*(x+1)) for x in range(25)]
    tenth_list = []
    for idx, ten in enumerate(tenths):
        col_tenth = col[idx:ten]
        col_dict = {}
        for letter in col_tenth:
            if letter in col_dict:
                col_dict[letter] += 1
            else:
                col_dict[letter] = 1
        for letter in letters:
            if letter not in col_dict:
                col_dict[letter] = 0
        for key, value in col_dict.items():
            col_dict[key] = value/(length/25)
        tenth_list.append(col_dict)
    labels = np.arange(0, 104, 4)
    for idx, (t, w) in enumerate(zip(tenth_list, wild_list)):
        for l in letters:
            if (w[l] != 0) and (t[l] != 0):
                train[f'{labels[idx]}-{labels[idx+1]}{l}_count'] = (t[l] - w[l])/(w[l] + t[l])
            else: 
                train[f'{labels[idx]}-{labels[idx+1]}{l}_count'] = 0
    return train
        
    
        
        

In [96]:
df['seq_len'] = df['protein_sequence'].str.len()
df['seq_len_norm'] = df['seq_len']/df['seq_len'].max()

In [97]:
df['len_tenth'] = df['seq_len']//25
wild_tenth = len(wild_type)//25
wild_tenths = []
tenth_labels = []
labels = np.arange(0, 25)
for label in labels:
    df[f'{label}_tenth_str'] = df.apply(partial(tenth_strings, c='protein_sequence', num=label, tenth='len_tenth'), axis=1)
    tenth_labels.append(f'{label}_tenth_str')
    wild_tenths.append(wild_type[label*wild_tenth: (label+1)*wild_tenth])
    
for lab, wild in zip(tenth_labels, wild_tenths):
    df[f'{lab}_match'] = df.apply(partial(similar, c=lab, w=wild, b=0, side=None), axis=1)

In [98]:
col = wild_type
length = len(col)
tenths = [int((length/25)*(x+1)) for x in range(25)]
wild_list = []
for idx, ten in enumerate(tenths):
    col_tenth = col[idx:ten]
    col_dict = {}
    for letter in col_tenth:
        if letter in col_dict:
            col_dict[letter] += 1
        else:
            col_dict[letter] = 1
    for letter in letters:
        if letter not in col_dict:
            col_dict[letter] = 0
    for key, value in col_dict.items():
        col_dict[key] = value/(length/25)
    wild_list.append(col_dict)

In [99]:
df = df.apply(partial(tenths_letters, letters=letters), axis=1)

In [100]:
df = match_columns(wild_type, df)

In [101]:
df['rel_stab'] = df['tm']/df['tm'].max()

In [102]:
rel_stab = df['rel_stab']

In [103]:
removals = ['seq_id', 'protein_sequence', 'pH', 'data_source', 'tm', 'seq_len', 'len_tenth', '0_tenth_str', '1_tenth_str', '2_tenth_str', '3_tenth_str', '4_tenth_str', '5_tenth_str', '6_tenth_str', '7_tenth_str', '8_tenth_str', '9_tenth_str', 'rel_stab', 'seq_len', 'seq_len_norm']
columns = df.columns
for col in columns:
    if col[-3:] == 'str':
        removals.append(col)
df = df.drop(removals, axis=1)

In [104]:
print(df)

       0_tenth_str_match  1_tenth_str_match  2_tenth_str_match  \
0               0.000000           0.190476           0.190476   
1               0.210526           0.210526           0.210526   
2               0.074074           0.148148           0.222222   
3               0.000000           0.111111           0.111111   
4               0.181818           0.151515           0.121212   
...                  ...                ...                ...   
31385           0.000000           0.068966           0.275862   
31386           0.153846           0.230769           0.307692   
31387           0.000000           0.307692           0.461538   
31388           0.064516           0.387097           0.258065   
31389           0.206897           0.275862           0.206897   

       3_tenth_str_match  4_tenth_str_match  5_tenth_str_match  \
0               0.380952           0.285714           0.285714   
1               0.315789           0.210526           0.210526   
2        

In [105]:
df.to_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/variables_6.csv')
rel_stab.to_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/target_2.csv')